# Introduction

Welcome to the Amazon Nova Reel Workshop! In this hands-on session, we'll explore the powerful capabilities of Amazon's video generation foundation model. You'll experience how **OctankFashion**, a fictional clothing brand, uses the model to create videos for use in their marketing and advertising campaigns.

## Use Case

OctankFashion would like to incorporate more rich video content into their marketing and advertising. They find traditional video production to be both too time-consuming and cost prohibitive, having to rent equipment, hire a video production agency, and pay for travel to on-location shoots. They will apply Nova Reel to tasks including:

1. Creating an evocative lifestyle video of their new line of summer t-shirts.
2. Turning one of their existing marketing images into full motion video.

## Workshop Objectives

By the end of this workshop, you will:

1. Understand the key features of Amazon Nova Reel model.
2. Learn how to use these features for marketing and advertising use cases.
3. Gain hands-on experience with the Amazon Bedrock API for video generation tasks.

## Features We'll Use

During the workshop, we'll leverage the following features of Amazon Nova Reel:

1. Text-to-Video
2. Image-to-Video

<div class="alert alert-block alert-warning">
<b>Prerequisites:</b> Please run the prerequiresites <b>00-prerequisites.ipynb</b> first before proceeding.
</div>


### Introduction to text-to-video

With Nova Reel's text-to-video capability, you can create a video from a simple text prompt describing your intended subject, environment, camera motion, and more.

Run the cell below to create an instance of the Bedrock Runtime client which we'll use later.


In [15]:
import boto3
import video_gen_util

boto3.setup_default_session(region_name="us-east-1")
session = boto3.Session()
sts_client = session.client("sts")

bedrock_runtime = boto3.client("bedrock-runtime")

#### Setting up storage

Generating a video takes some time - approximately 3-4 minutes to produce a 6 second video. To accomodate this execution time, the Bedrock Runtime introduces a new asynchronous invocation API. You start generating a video by calling the `start_async_invoke()` method on the Bedrock Runtime client. When the generation job completes, Bedrock automatically saves the generated video to an S3 bucket you specify.

Run the cell below to automatically create a new S3 bucket in your account which we will use as a destination for your videos. (If you already have an existing bucket you'd like to use, edit the `s3_destination_bucket` variable to use that bucket's ID)


In [3]:
region = session.region_name
account_id = sts_client.get_caller_identity()["Account"]

create_new_bucket = False

if create_new_bucket == True:
    new_bucket_name = f"ovg-bucket-{region}-{account_id}"
    s3_destination_bucket = new_bucket_name
    # Create the bucket
    boto3.client("s3").create_bucket(Bucket=s3_destination_bucket)
else:
    # Replace this with an existing bucket ID if you'd like.
    s3_destination_bucket = 'sagemaker-us-west-2-710299592439/genai/reel/'


#### Example 1: Generate a video with a text prompt

OctankFashion designed their new line of graphic t-shirts to be perfect for summer. The want a video asset that evokes the feeling of a tranquil day and draws a strong connection between that feeling and their apparel. With Nova Reel's text-to-video feature, it's easy to create any video you can dream of.

Nova Reel supports a single task type called "TEXT_VIDEO". The following parameter is relevant to using that task type for text-to-video. The parameter is encapsulated in the `textToVideoParams` field of the request body.

- `text` (Optional) – A text prompt describing your desired video. Must be 1 - 512 characters in length.

Run the cells below to start generating the video.


In [4]:
# Define the main input parameters.
text = "fabric shirts ripple in the strong breeze, white collared t-shirts with black palm tree graphics in their centers hang rippling on a clothes line between two real palm trees. Lush greenery in the foreground opens to a beautiful beach and waves in the background. Soft sunlight."

seed = 1841511189

In [5]:
import json

model_input = {
    "taskType": "TEXT_VIDEO",
    "textToVideoParams": {"text": text},
    "videoGenerationConfig": {
        "durationSeconds": 6,  # 6 is the only supported value currently.
        "fps": 24,  # 24 is the only supported value currently.
        "dimension": "1280x720",  # "1280x720" is the only supported value currently.
        "seed": seed,  # Any number from 0 through 2,147,483,646
    },
}

# Start the asynchronous video generation job.
invocation = bedrock_runtime.start_async_invoke(
    modelId="amazon.nova-reel-v1:0",
    modelInput=model_input,
    outputDataConfig={"s3OutputDataConfig": {"s3Uri": f"s3://{s3_destination_bucket}"}},
)

# This will be used by other cells in this notebook.
invocation_arn = invocation["invocationArn"]

# Pretty print the response JSON.
print("\nResponse:")
print(json.dumps(invocation, indent=2, default=str))

# Save the invocation details for later reference. Helpful for debugging and reporting feedback.
video_gen_util.save_invocation_info(invocation, model_input)


Response:
{
  "ResponseMetadata": {
    "RequestId": "cb41c7c3-ab13-4b15-856b-8ed6a4b5f578",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "date": "Fri, 07 Feb 2025 07:39:33 GMT",
      "content-type": "application/json",
      "content-length": "84",
      "connection": "keep-alive",
      "x-amzn-requestid": "cb41c7c3-ab13-4b15-856b-8ed6a4b5f578"
    },
    "RetryAttempts": 0
  },
  "invocationArn": "arn:aws:bedrock:us-east-1:710299592439:async-invoke/g2rd27o9bikk"
}


'/home/sagemaker-user/amazon-nova-samples/multimodal-generation/workshop-sample/NovaReel/output/2025-02-07_07-39-32_g2rd27o9bikk'

#### Checking the status of generation jobs

While your video is generating, you can check its status in one of two ways.

##### Method 1: Using `get_async_invoke()`

If you know the invocation ARN of the job you'd like to check, you can use the the `get_async_invoke()` method to check the job's status. Run the cell below to check the status of the last job you initiated above.


In [6]:
response = bedrock_runtime.get_async_invoke(invocationArn=invocation_arn)

status = response["status"]
print(f"Status: {status}")

# Pretty print the JSON response
print("\nFull response:")
print(json.dumps(response, indent=2, default=str))

Status: InProgress

Full response:
{
  "ResponseMetadata": {
    "RequestId": "ec8f600c-5559-4846-b913-8bd330e7b5ff",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "date": "Fri, 07 Feb 2025 07:39:54 GMT",
      "content-type": "application/json",
      "content-length": "439",
      "connection": "keep-alive",
      "x-amzn-requestid": "ec8f600c-5559-4846-b913-8bd330e7b5ff"
    },
    "RetryAttempts": 0
  },
  "invocationArn": "arn:aws:bedrock:us-east-1:710299592439:async-invoke/g2rd27o9bikk",
  "modelArn": "arn:aws:bedrock:us-east-1::foundation-model/amazon.nova-reel-v1:0",
  "clientRequestToken": "0bbd1c18-4153-44b4-9e89-d811a5543f88",
  "status": "InProgress",
  "submitTime": "2025-02-07 07:39:32+00:00",
  "lastModifiedTime": "2025-02-07 07:39:32+00:00",
  "outputDataConfig": {
    "s3OutputDataConfig": {
      "s3Uri": "s3://sagemaker-us-west-2-710299592439/genai/reel/g2rd27o9bikk"
    }
  }
}


##### Method 2: Using `list_async_invokes()`

The following code demonstrates how to check the status of **all** video generation jobs in your account using the `list_async_invokes()` method. This method is useful when you don't know the invocation ARN of a job you want to check the status of or if you want to create a dashboard that lists running, completed, or failed jobs. This method supports a number of arguments that allow you to filter by status, date, and a `maxResults` limit. It also supports pagination. We don't show the full API in this example.


In [7]:
invocation = bedrock_runtime.list_async_invokes(
    maxResults=10,  # (Optional)
    statusEquals="InProgress",  # (Optional) Can be "Completed", "InProgress", or "Failed". Omit this argument to list all jobs, regardless of status.
    # Note: There are other supported arguments not demonstrated here.
)

# Pretty print the JSON response
print("Invocation Jobs:")
print(json.dumps(invocation, indent=2, default=str))

Invocation Jobs:
{
  "ResponseMetadata": {
    "RequestId": "97d19edf-618e-4cb6-bfb9-b66901688cd5",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "date": "Fri, 07 Feb 2025 07:40:41 GMT",
      "content-type": "application/json",
      "content-length": "466",
      "connection": "keep-alive",
      "x-amzn-requestid": "97d19edf-618e-4cb6-bfb9-b66901688cd5"
    },
    "RetryAttempts": 0
  },
  "asyncInvokeSummaries": [
    {
      "invocationArn": "arn:aws:bedrock:us-east-1:710299592439:async-invoke/g2rd27o9bikk",
      "modelArn": "arn:aws:bedrock:us-east-1::foundation-model/amazon.nova-reel-v1:0",
      "clientRequestToken": "0bbd1c18-4153-44b4-9e89-d811a5543f88",
      "status": "InProgress",
      "submitTime": "2025-02-07 07:39:32+00:00",
      "lastModifiedTime": "2025-02-07 07:40:20+00:00",
      "outputDataConfig": {
        "s3OutputDataConfig": {
          "s3Uri": "s3://sagemaker-us-west-2-710299592439/genai/reel/g2rd27o9bikk"
        }
      }
    }
  ]
}


##### Download and view the generated videos

We've provided a set of utility functions in the `video_gen_util.py` script. One of these functions provides one solution to automatically downloading previously completed jobs and monitoring in-progress jobs. Finished jobs will be automatically downloaded to the "output" folder.


In [9]:
# Check CMI job status
while True:
    response = bedrock_runtime.get_async_invoke(invocationArn=invocation_arn)
    status = response["status"]
    print(f"Status: {status}")
    
    if status in ['Completed', 'Failed']:
        break
        
    time.sleep(60)  # Check every 60 seconds

Status: Completed


In [31]:
import boto3
from botocore.exceptions import ClientError
from urllib.parse import urlparse

def parse_s3_uri_with_urllib(s3_uri):
    """
    Parse an S3 URI using urllib
    
    Args:
        s3_uri (str): S3 URI in format 's3://bucket-name/prefix/path'
        
    Returns:
        tuple: (bucket_name, prefix)
    """
    parsed = urlparse(s3_uri)
    bucket_name = parsed.netloc
    prefix = parsed.path.lstrip('/')
    
    return bucket_name, prefix
    
def download_file(s3Uri, file_name, local_path):
    """
    Download a file from S3 using prefix/key
    """
    s3_client = boto3.client('s3')
    bucket_name, prefix = parse_s3_uri_with_urllib(s3Uri)
    print(f"\nURI: {s3Uri}, Bucket: {bucket_name}, Prefix: {prefix}")
    try:
        s3_client.download_file(
            Bucket=bucket_name,
            Key=f"{prefix}/{file_name}",
            Filename=local_path
        )
        print(f"Successfully downloaded {file_name} to {local_path}")
    except ClientError as e:
        print(f"Error downloading file: {e}")
        return False
    return True


In [35]:
import os

completed_jobs_args = {"statusEquals": "Completed"}
completed_jobs = bedrock_runtime.list_async_invokes(**completed_jobs_args)

for job in completed_jobs["asyncInvokeSummaries"]:
    job_id = video_gen_util.get_job_id_from_arn(job["invocationArn"])
    print(f"try to download completed job {job_id}.")
    output_s3_uri = (
        job["outputDataConfig"]["s3OutputDataConfig"]["s3Uri"]
    )
    print(f"The asyncInvokeSummaries s3Uri {output_s3_uri}")
    #video_gen_util.save_completed_job(job)
    output_folder = "ouput"
    output_folder_abs = os.path.abspath(
        f"{output_folder}/{video_gen_util.get_folder_name_for_job(job)}"
    )
    # Ensure the output folder exists
    os.makedirs(output_folder_abs, exist_ok=True)
    output_filename = "output.mp4"
    local_file_path = os.path.join(output_folder_abs, output_filename)
    print(f"save file to {local_file_path}")
    
    download_file(output_s3_uri, output_filename, local_file_path)

try to download completed job g2rd27o9bikk.
The asyncInvokeSummaries s3Uri s3://sagemaker-us-west-2-710299592439/genai/reel/g2rd27o9bikk
save file to /home/sagemaker-user/amazon-nova-samples/multimodal-generation/workshop-sample/NovaReel/ouput/2025-02-07_07-39-32_g2rd27o9bikk/output.mp4

URI: s3://sagemaker-us-west-2-710299592439/genai/reel/g2rd27o9bikk, Bucket: sagemaker-us-west-2-710299592439, Prefix: genai/reel/g2rd27o9bikk
Successfully downloaded output.mp4 to /home/sagemaker-user/amazon-nova-samples/multimodal-generation/workshop-sample/NovaReel/ouput/2025-02-07_07-39-32_g2rd27o9bikk/output.mp4


In [17]:
from datetime import datetime, timedelta, timezone

# Download and monitor videos from the past N hours.
duration_hours = 2

from_submit_time = datetime.now(timezone.utc) - timedelta(hours=duration_hours)
video_gen_util.monitor_and_download_videos("output", submit_time_after=from_submit_time)

Problem: No MP4 file was found in S3 at sagemaker-us-west-2-710299592439/g2rd27o9bikk
Monitoring 0 "InProgress" jobs.
Monitoring and download complete!


Now, check the `output` folder in your Sagemaker, and you can view the video files in `.mp4` format.


## Take Away

Amazon Nova Reel's text-to-image feature makes it easy and cost-effective for anyone to create engaging video content of anything they can imagine from just a simple text prompt. Through hands-on experience with the Amazon Bedrock API, you learned to generate compelling visual content for real-world scenarios, such as showcasing fashion items. Now, to learn how to get even more control over the video output, move on to the "02-image-to-video.ipynb" notebook.
